In [2]:

import time
import os 
import json
import numpy as np
import pandas as pd
import lxml.html as html
import scrapper_utils as scu
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from selenium.webdriver.support.ui import Select
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

chromium_path =  "drivers/chrome-linux64/chrome"
service_path = "drivers/chromedriver-linux64/chromedriver"

url = "https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/{state}/distrito/{district}/secciones/todas" 
XPATH = {
    'XPATH_STATES' : '//select[@id="select-entidad"]',
    'XPATH_DISTRICT' : '//select[@id="select-distriro"]',
    'XPATH_PART' : '/html/body/app-root/app-federal/div/div/div[3]/app-entidad/div/app-porcentaje-participacion/div/div/div/div/div[2]/div[2]/p',
}

In [3]:
with open('general/states_districts.json') as f:
    states_districts = json.load(f)
    print(states_districts)

{'Aguascalientes': ['1. Jesus Maria', '2. Aguascalientes', '3. Aguascalientes'], 'Baja California': ['1. Mexicali', '2. Mexicali', '3. Ensenada', '4. Tijuana', '5. Tijuana', '6. Tijuana', '7. Mexicali', '8. Tijuana', '9. Tecate'], 'Baja California Sur': ['1. La Paz', '2. San Jose Del Cabo'], 'Campeche': ['1. San Francisco de Campeche', '2. Ciudad Del Carmen'], 'Coahuila': ['1. Piedras Negras', '2. San Pedro', '3. Monclova', '4. Saltillo', '5. Torreon', '6. Torreon', '7. Saltillo', '8. Ramos Arizpe'], 'Colima': ['1. Colima', '2. Valle de Las Garzas'], 'Chiapas': ['1. Palenque', '2. Bochil', '3. Ocosingo', '4. Pichucalco', '5. San Cristobal de Las Casas', '6. Tuxtla Gutierrez', '7. Tonala', '8. Comitan de Dominguez', '9. Tuxtla Gutierrez', '10. Villaflores', '11. Las Margaritas', '12. Tapachula', '13. Huehuetan'], 'Chihuahua': ['1. Juarez', '2. Juarez', '3. Juarez', '4. Juarez', '5. Delicias', '6. Chihuahua', '7. Cuauhtemoc', '8. Chihuahua', '9. Hidalgo Del Parral'], 'Ciudad De México': 

In [ ]:
with open('general/states_dict.json') as f:
    states_dict = json.load(f)
    print(states_dict)

In [32]:
s = Service(service_path)
chrome_options = Options()
chrome_options.binary_location = chromium_path
file_path = 'districts'
district_votes_path = f'{file_path}/district_votes'
scu.create_folder_if_not_exists(district_votes_path)
district_section_path = f'{file_path}/district_section'
scu.create_folder_if_not_exists(district_section_path) 

for state_id, (state_name, state_district) in enumerate(states_districts.items()):
    print(f'Scrapping {state_name} with id {state_id+1}')
    for district_id, district_name in enumerate(state_district):
        scrap_url = url.format(state=state_id+1,district=district_id+1)
        print(f'Scrapping {district_name} with id {district_id+1} #and url = {scrap_url}')
        with webdriver.Chrome(service=s, options=chrome_options) as driver:
            driver.get(scrap_url)
            driver.maximize_window()
            scu.get_info_district_votes_in_CR(driver).to_csv(f'{district_votes_path}/{state_name}_{state_id+1}.csv',encoding='utf-8-sig')
            scu.json_to_dataframe(scu.get_info_district_section(driver)).to_csv(f'{district_section_path}/{state_name}_{state_id+1}.csv',encoding='utf-8-sig')
            driver.close()



The folder 'districts/district_votes' has been created.
The folder 'districts/district_section' has been created.
Scrapping Aguascalientes with id 1
Scrapping 1. Jesus Maria with id 1 #and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/1/distrito/1/secciones/todas
Scrapping 2. Aguascalientes with id 2 #and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/1/distrito/2/secciones/todas
Scrapping 3. Aguascalientes with id 3 #and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/1/distrito/3/secciones/todas
Scrapping Baja California with id 2
Scrapping 1. Mexicali with id 1 #and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/2/distrito/1/secciones/todas
Scrapping 2. Mexicali with id 2 #and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/2/distrito/2/secciones/todas
Scrapping 3. Ensenada with id 3 #and url = https://prep2024.ine.mx/pub

In [22]:
s = Service(service_path)
chrome_options = Options()
chrome_options.binary_location = chromium_path

with webdriver.Chrome(service=s, options=chrome_options) as driver:
    driver.get(url.format(state=1,district=3))
    driver.maximize_window()
    df = scu.get_info_district_votes_in_CR(driver)
    driver.close()


In [23]:
df

,Sección,PAN,PRI,PRD,V,PT,MC,M,PA+PRI+PRD,PAN+PRI,PAN+PRD,PRI+PRD,V+PT+M,V+PT,V+M,PT+M,Candidaturas no registradas,Votos nulos,Total
0,Voto Anticipado,54,8,1,3,1,0,16,0,0,0,0,0,0,0,0,4,0,87
1,Voto en Prisión Preventiva,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Sección 1,1546,84,16,11,7,17,96,51,12,3,0,7,0,2,1,4,13,1870
3,Sección 2,184,15,2,6,2,8,42,2,2,0,0,1,0,0,0,1,5,270
4,Sección 3,700,56,10,19,11,98,292,31,9,2,0,25,0,3,3,5,14,1278
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299,Sección 679,192,39,9,14,17,17,137,5,7,1,0,9,2,0,3,0,13,465
300,Sección 680,444,57,15,42,33,191,680,24,9,0,0,42,1,6,4,2,30,1580
301,Sección 681,199,42,1,12,6,25,50,7,5,0,0,3,0,2,0,0,15,367
302,Sección 682,916,94,21,37,37,224,574,64,10,0,0,40,0,3,8,10,31,2069


In [3]:
s = Service(service_path)
chrome_options = Options()
chrome_options.binary_location = chromium_path

with webdriver.Chrome(service=s, options=chrome_options) as driver:
    driver.get(url.format(state=1,district=3))
    driver.maximize_window()
    df = scu.get_info_district_section(driver)
    driver.close()

In [6]:
scu.json_to_dataframe(df)

,Total de votos,Porcentaje
PAN,105259,0.450471
PRI,10994,0.047050
PRD,2567,0.010985
V,4798,0.020533
PT,4244,0.018162
MC,18938,0.081048
M,68008,0.291050
PA+PRI+PRD,5040,0.021569
PAN+PRI,1411,0.006038
PAN+PRD,180,0.000770


In [4]:
s = Service(service_path)
chrome_options = Options()
chrome_options.binary_location = chromium_path
main_url = 'https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/{state}/candidatura'
with webdriver.Chrome(service=s, options=chrome_options) as driver:
    print(main_url.format(state=1))
    driver.get(main_url.format(state=1))
    driver.maximize_window()
    states = scu.get_states_list(driver,XPATH['XPATH_STATES'])
    # print(states)
    driver.close()

https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/1/candidatura


In [12]:
states_dict = scu.get_states_json(states)
file_path = 'general'
scu.create_folder_if_not_exists(file_path)
scu.create_json_file(states_dict, f'{file_path}/states_dict')

The folder 'general' has been created.


In [5]:
!ls -la

total 524
drwxrwxr-x 8 thinkpad thinkpad   4096 jun  4 17:49 .
drwxrwxr-x 6 thinkpad thinkpad   4096 jun  3 10:27 ..
-rw-rw-r-- 1 thinkpad thinkpad 372800 jun  4 14:03 District.ipynb
drwxrwxrwx 4 root     root       4096 jun  3 10:29 drivers
drwxrwxr-x 2 thinkpad thinkpad   4096 jun  4 17:49 general
-rw-rw-r-- 1 thinkpad thinkpad  36214 jun  4 17:55 get_info_by_district.ipynb
-rw-rw-r-- 1 thinkpad thinkpad  33767 jun  4 16:28 get_info_by_states.ipynb
drwxrwxr-x 2 thinkpad thinkpad   4096 jun  4 17:35 __pycache__
-rwxrwxr-x 1 thinkpad thinkpad  10370 jun  4 17:54 scrapper_utils.py
drwxrwxr-x 2 thinkpad thinkpad   4096 jun  4 14:23 states
drwxrwxr-x 2 thinkpad thinkpad   4096 jun  4 14:47 states2
drwxrwxr-x 2 thinkpad thinkpad   4096 jun  4 15:03 states3
-rw-rw-r-- 1 thinkpad thinkpad  27684 jun  4 14:03 States.ipynb
-rw-rw-r-- 1 thinkpad thinkpad   8414 jun  4 17:55 test.ipynb


In [3]:
import json
with open('general/states_dict.json') as f:
    states_dict = json.load(f)
    print(states_dict)

{'0': 'Aguascalientes', '1': 'Baja California', '2': 'Baja California Sur', '3': 'Campeche', '4': 'Coahuila', '5': 'Colima', '6': 'Chiapas', '7': 'Chihuahua', '8': 'Ciudad De México', '9': 'Durango', '10': 'Guanajuato', '11': 'Guerrero', '12': 'Hidalgo', '13': 'Jalisco', '14': 'México', '15': 'Michoacán', '16': 'Morelos', '17': 'Nayarit', '18': 'Nuevo León', '19': 'Oaxaca', '20': 'Puebla', '21': 'Querétaro', '22': 'Quintana Roo', '23': 'San Luis Potosí', '24': 'Sinaloa', '25': 'Sonora', '26': 'Tabasco', '27': 'Tamaulipas', '28': 'Tlaxcala', '29': 'Veracruz', '30': 'Yucatán', '31': 'Zacatecas'}


In [5]:
main_url = 'https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/{state}/candidatura'

s = Service(service_path)
chrome_options = Options()
chrome_options.binary_location = chromium_path
states_districs = scu.get_states_districs_json(states_dict, XPATH['XPATH_DISTRICT'], main_url, s, chrome_options)
states_districs


Scrapping Aguascalientes with id 0 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/1/candidatura
Scrapping Baja California with id 1 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/2/candidatura
Scrapping Baja California Sur with id 2 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/3/candidatura
Scrapping Campeche with id 3 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/4/candidatura
Scrapping Coahuila with id 4 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/5/candidatura
Scrapping Colima with id 5 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/6/candidatura
Scrapping Chiapas with id 6 and url = https://prep2024.ine.mx/publicacion/nacional/presidencia/nacional/entidad/7/candidatura
Scrapping Chihuahua with id 7 and url = https://prep2024.ine.mx/publicacion/nacional/presi

{'Aguascalientes': ['1. Jesus Maria',
  '2. Aguascalientes',
  '3. Aguascalientes'],
 'Baja California': ['1. Mexicali',
  '2. Mexicali',
  '3. Ensenada',
  '4. Tijuana',
  '5. Tijuana',
  '6. Tijuana',
  '7. Mexicali',
  '8. Tijuana',
  '9. Tecate'],
 'Baja California Sur': ['1. La Paz', '2. San Jose Del Cabo'],
 'Campeche': ['1. San Francisco de Campeche', '2. Ciudad Del Carmen'],
 'Coahuila': ['1. Piedras Negras',
  '2. San Pedro',
  '3. Monclova',
  '4. Saltillo',
  '5. Torreon',
  '6. Torreon',
  '7. Saltillo',
  '8. Ramos Arizpe'],
 'Colima': ['1. Colima', '2. Valle de Las Garzas'],
 'Chiapas': ['1. Palenque',
  '2. Bochil',
  '3. Ocosingo',
  '4. Pichucalco',
  '5. San Cristobal de Las Casas',
  '6. Tuxtla Gutierrez',
  '7. Tonala',
  '8. Comitan de Dominguez',
  '9. Tuxtla Gutierrez',
  '10. Villaflores',
  '11. Las Margaritas',
  '12. Tapachula',
  '13. Huehuetan'],
 'Chihuahua': ['1. Juarez',
  '2. Juarez',
  '3. Juarez',
  '4. Juarez',
  '5. Delicias',
  '6. Chihuahua',
  '7

In [6]:
file_path = 'general'
scu.create_folder_if_not_exists(file_path)
scu.create_json_file(states_districs, f'{file_path}/states_districts')

The folder 'general' already exists.
